# Plots by depth

Evaluate the distribution and attributes of features by depth.

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 0. Import data and define constants

In [8]:
picks = pd.read_csv('1_Pre-processed data.csv')
picks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   depth_m      450 non-null    float64
 1   dip_az       450 non-null    float64
 2   dip          450 non-null    float64
 3   aperture_mm  450 non-null    float64
 4   type         450 non-null    object 
 5   strike       450 non-null    float64
 6   lithology    442 non-null    object 
 7   depth_ft     450 non-null    float64
dtypes: float64(6), object(2)
memory usage: 28.2+ KB


In [9]:
picks.type.unique()

array(['Conductive, HC', 'Conductive, LC', 'Bed boundary'], dtype=object)

In [11]:
fractures = picks[(picks['type'] == 'Conductive, HC') | (picks['type'] == 'Conductive, LC')]
fractures.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 249
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   depth_m      250 non-null    float64
 1   dip_az       250 non-null    float64
 2   dip          250 non-null    float64
 3   aperture_mm  250 non-null    float64
 4   type         250 non-null    object 
 5   strike       250 non-null    float64
 6   lithology    242 non-null    object 
 7   depth_ft     250 non-null    float64
dtypes: float64(6), object(2)
memory usage: 17.6+ KB


In [12]:
beds = picks[(picks['type'] == 'Bed boundary')]
beds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 250 to 449
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   depth_m      200 non-null    float64
 1   dip_az       200 non-null    float64
 2   dip          200 non-null    float64
 3   aperture_mm  200 non-null    float64
 4   type         200 non-null    object 
 5   strike       200 non-null    float64
 6   lithology    200 non-null    object 
 7   depth_ft     200 non-null    float64
dtypes: float64(6), object(2)
memory usage: 14.1+ KB


In [14]:
# Use the picks DataFrame to define log top and bottom depths

top_m = picks['depth_m'].min() - 1
bot_m = picks['depth_m'].max() + 1

## 1. Use Seaborn to evaluate pick attributes by depth

## 3. Calculate frequency histograms

In [15]:
def pick_histogram_param(depths, log_top, log_base, count_interval):
    """
    Calculate parameters for plotting frequency histograms where 
    the average number of picks per unit length is calculated for each bin.

    Parameters:
    - depths (array-like): A sequence of depth values for which to calculate parameters.
    - log_top (float): The top (shallowest) depth of the log. Must be above the shallowest pick.
    - log_base (float): The base (deepest) depth of the log. Must be below the deepest pick.
    - count_interval (float): The desired interval for histogram bin width.
    - bw_method_input (float, optional): Bandwidth parameter for KDE (default: 0.1).

    Returns:
    - tuple: A tuple containing the following parameters:
        - bin_mids (array): Midpoints of histogram bins.
        - bin_width (float): Width of histogram bins.
        - counts (array): Counts of data points in each histogram bin.
        - counts_per_bin (array): Counts per unit length for each bin.

    Notes:
    - This function calculates parameters for creating histograms where bin width
      is controlled by log length, ensuring consistent binning for varying log depths.
    - The `count_interval` parameter controls the width of histogram bins in linear space.
    """
    n_bins = round((log_base - log_top) / count_interval)
    if n_bins < 3: n_bins = 3

    counts,bin_edges = np.histogram(depths, bins = n_bins)
    bin_mids = 0.5 * (bin_edges[:-1] + bin_edges[1:])
    bin_width = bin_edges[1] - bin_edges[0]
    counts_per_bin = counts/bin_width

    return bin_mids, bin_width, counts, counts_per_bin


In [ ]:
# calculate values for histogram
depth_values = fractures.depth_ft.values
bin_mids, bin_width, counts, counts_per_bin, kernel = pick_histogram_param(
    depth_values,
    top_m, 
    bot_m, 
    10, # bin length
    bw_method_input=0.1)

fig, ax1 = plt.subplots(1,1,figsize=(2,12), sharey=True)

plt.title('Elec Conductive Frac')

ax1.barh(
    bin_mids, 
    counts_per_bin, 
    bin_width,
    color='k',
    alpha=0.2,
    label='hist raw'
    )

ax1.set_ylim(bot_depth_ft,top_depth_ft)
ax1.set_ylabel('Depth [ft-MDRF]')
ax1.xaxis.set_ticks([0,0.5,1,1.5])

ax1.grid(alpha=0.3)
ax1.set_xlabel('Ave frac/ft')

# this locator puts ticks at regular intervals
loc = plticker.MultipleLocator(base=100) 
ax1.yaxis.set_major_locator(loc)

#ax1.axes.get_yaxis().set_ticklabels([])


plt.savefig(
    'Plots/River Ranch 20__3_frequency_plots_HAL_ElecConductiveFrac.png', 
    dpi=400, 
    bbox_inches='tight', 
    transparent=True
    )